# 0.Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
import seaborn as sns

In [3]:
import pandas as pd
import joblib
import warnings;warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

In [4]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

# 1.Data Load

In [5]:
cus = pd.read_csv("../data/cus_info.csv")
iem = pd.read_csv("../data/iem_info.csv")
hist = pd.read_csv("../data/stk_bnc_hist.csv")
train = pd.read_csv("../data/stk_hld_train.csv")
test = pd.read_csv("../data/stk_hld_test.csv")
submission = pd.read_csv("../data/sample_submission.csv")
df_date = pd.read_csv('../data/날짜별_개장여부.csv',encoding='cp949')
sell_dt = pd.read_csv('../data/tk_hld_newtrain.csv')

In [6]:
#cus.columns = ['계좌ID','성별','연령대','투자성향','자산구간','주거래상품군','Life Style','서비스등급','총 투자기간','주거래업종구분']
display(cus.head())
print(cus.shape)

,act_id,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd
0,64aae8dd71e5c0761000db9f9a6779e504e7a4aa9dc097...,1,4,99,1,2,3,3,6,16
1,5f7c3a8f37d9c268d06130ff0be5d32a1b9ef68c13049f...,1,6,4,4,2,5,2,6,1
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2,7,4,3,2,5,5,6,9
3,6d497facfa1ea5901b827335553331f8555fec02a8184f...,2,6,4,4,8,5,3,6,16
4,b727c78d2cfc246c97b677f29a034399a0c7e7873fff44...,1,5,2,2,2,5,5,5,16


(10000, 10)


In [7]:
#iem.columns = ['종목코드','종목한글명','종목업종','시가총액,규모유형','시장구분']
display(iem.head())
print(iem.shape)

,iem_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,A000020,동화약품,14,2,99
1,A000080,하이트진로,14,2,1
2,A000180,성창기업지주,14,3,99
3,A000227,유유제약2우B,14,99,99
4,A000325,노루홀딩스우,14,99,99


(3078, 5)


In [8]:
#hist.columns = ['계좌ID','기준일자','종목코드','잔고수량','잔고금액','액면가']
display(hist.head())
print(hist.shape)

,act_id,bse_dt,iem_cd,bnc_qty,tot_aet_amt,stk_par_pr
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200820,A008770,40.0,2828000.0,5000.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A008770,20.0,1390000.0,5000.0
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20160104,A005940,311.0,2982490.0,5000.0
3,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200814,A005930,40.0,2320000.0,100.0
4,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A005930,20.0,1028000.0,100.0


(2573839, 6)


In [9]:
#train.columns = ['계좌ID','종목코드','매수일자','보유기간(일)']
display(train.head())
print(train.shape)

,act_id,iem_cd,byn_dt,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324


(681472, 4)


In [10]:
#test.columns = ['계좌ID','종목코드','매수일자','과거보유일','제출ID','보유기간(일)']
display(test.head())
print(test.shape)

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0


(70596, 6)


In [11]:
display(submission.head())
print(submission.shape)

,submit_id,hold_d
0,IDX00001,0
1,IDX00002,0
2,IDX00003,0
3,IDX00004,0
4,IDX00005,0


(70596, 2)


In [12]:
display(sell_dt.head())
print(sell_dt.shape)

,Unnamed: 0,act_id,iem_cd,byn_dt,hold_d,sell_dt
0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,2018-08-10
1,1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,80,2018-05-31
2,2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,5,2018-05-25
3,3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,22,2020-12-14
4,4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,324,2020-01-03


(681472, 6)


In [13]:
df_date['날짜'] = pd.to_datetime(df_date['날짜'],format='%Y-%m-%d')
display(df_date.head())
print(df_date.shape)

,날짜,개장
0,2016-01-01,0
1,2016-01-02,0
2,2016-01-03,0
3,2016-01-04,1
4,2016-01-05,1


(2192, 2)


# 2.Data Merge

In [14]:
# 앞서 언급한 것처럼 베이스라인에서는 "hist_d" column을 임의로 생성하여 모델을 학습시키도록 하겠습니다.
# 베이스라인에서는 "hold_d"값, 즉 주식 보유기간의 0.6배에 해당하는 기간을 임의로 설정하여 "hist_d"를 생성하였습니다.
# 결국 모델은 "hist_d"만큼 주식을 보유 했을때의 "hold_d"를 예측하게 될 것입니다.

train["hist_d"] = train["hold_d"]*0.5 # 0.5 ~ 0.9
train.hist_d = np.trunc(train["hist_d"])

In [15]:
# train과 test에 고객정보(cus_info)와 주식정보(iem_info)를 추가하겠습니다.

train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

In [16]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)

In [17]:
# 추가적으로 약간의 전처리를 통해 train data와 test data를 구성하겠습니다.

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"] #주가당 금액?
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])] #이거는 왜 하는거지? 뜻: 매수일자 = 기준일자, 굳이
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

In [18]:
train_data

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,5.0,1,9,3,2,2,9,5,5,8,GS건설,14,1,1,20180726,274.0,11782000.0,5000.0,43000.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,40.0,1,9,3,2,2,9,5,5,8,삼성전자,14,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,2.0,1,9,3,2,2,9,5,5,8,코스모신소재,14,2,99,20180517,786.0,14619600.0,1000.0,18600.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,11.0,1,9,3,2,2,9,5,5,8,영진약품,14,2,1,20201112,60.0,462000.0,500.0,7700.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,162.0,1,9,3,2,2,9,5,5,8,아세아제지,14,3,99,20180905,183.0,8125200.0,5000.0,44400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A260660,20180831,0.0,1,4,4,2,2,3,4,3,8,알리코제약,14,3,99,20180831,199.0,3532250.0,500.0,17750.0
681468,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A271980,20201027,0.0,1,4,4,2,2,3,4,3,8,제일약품,14,2,99,20201027,488.0,22960400.0,500.0,47050.0
681469,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A289080,20181121,0.0,1,4,4,2,2,3,4,3,8,SV인베스트먼트,14,2,99,20181121,2210.0,9204650.0,500.0,4165.0
681470,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A307930,20200214,0.0,1,4,4,2,2,3,4,3,8,컴퍼니케이,14,3,99,20200214,285.0,2721750.0,500.0,9550.0


In [19]:
test_data

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,LG유플러스,14,1,1,20200522,300.0,3945000.0,5000.0,13150.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,이큐셀,14,3,99,20190823,198.0,2524500.0,500.0,12750.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,세틀뱅크,14,2,99,20200611,138.0,4291800.0,500.0,31100.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,파인텍,14,3,99,20200120,2972.0,6716720.0,500.0,2260.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,카카오게임즈,14,1,2,20201217,10.0,464500.0,100.0,46450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,5,2,5,3,5,2,삼성화재,14,1,1,20201127,1.0,198500.0,500.0,198500.0
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,5,2,5,3,5,2,NH투자증권우,14,99,99,20200731,50.0,372000.0,5000.0,7440.0
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,2,2,3,4,3,8,보루네오가구,14,3,99,20170704,75057.0,3677793.0,100.0,49.0
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,2,2,3,4,3,8,종근당홀딩스,14,2,99,20201215,96.0,11424000.0,2500.0,119000.0


In [20]:
L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

In [21]:
train_data

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,5.0,1,9,3,2,2,9,5,5,8,101,14,1,1,20180726,274.0,11782000.0,5000.0,43000.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,40.0,1,9,3,2,2,9,5,5,8,1361,14,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,2.0,1,9,3,2,2,9,5,5,8,2530,14,2,99,20180517,786.0,14619600.0,1000.0,18600.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,11.0,1,9,3,2,2,9,5,5,8,1968,14,2,1,20201112,60.0,462000.0,500.0,7700.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,162.0,1,9,3,2,2,9,5,5,8,1696,14,3,99,20180905,183.0,8125200.0,5000.0,44400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A260660,20180831,0.0,1,4,4,2,2,3,4,3,8,1752,14,3,99,20180831,199.0,3532250.0,500.0,17750.0
681468,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A271980,20201027,0.0,1,4,4,2,2,3,4,3,8,2345,14,2,99,20201027,488.0,22960400.0,500.0,47050.0
681469,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A289080,20181121,0.0,1,4,4,2,2,3,4,3,8,520,14,2,99,20181121,2210.0,9204650.0,500.0,4165.0
681470,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A307930,20200214,0.0,1,4,4,2,2,3,4,3,8,2460,14,3,99,20200214,285.0,2721750.0,500.0,9550.0


In [22]:
test_data

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,417,14,1,1,20200522,300.0,3945000.0,5000.0,13150.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,2231,14,3,99,20190823,198.0,2524500.0,500.0,12750.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,1514,14,2,99,20200611,138.0,4291800.0,500.0,31100.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,2681,14,3,99,20200120,2972.0,6716720.0,500.0,2260.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,2450,14,1,2,20201217,10.0,464500.0,100.0,46450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,5,2,5,3,5,2,1369,14,1,1,20201127,1.0,198500.0,500.0,198500.0
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,5,2,5,3,5,2,443,14,99,99,20200731,50.0,372000.0,5000.0,7440.0
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,2,2,3,4,3,8,1279,14,3,99,20170704,75057.0,3677793.0,100.0,49.0
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,2,2,3,4,3,8,2374,14,2,99,20201215,96.0,11424000.0,2500.0,119000.0


In [23]:
df = pd.concat([train_data,test_data]).reset_index(drop=True).iloc[:,:-2]
df

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,5.0,1,9,3,2,2,9,5,5,8,101,14,1,1,20180726,274.0,11782000.0,5000.0,43000.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,40.0,1,9,3,2,2,9,5,5,8,1361,14,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,2.0,1,9,3,2,2,9,5,5,8,2530,14,2,99,20180517,786.0,14619600.0,1000.0,18600.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,11.0,1,9,3,2,2,9,5,5,8,1968,14,2,1,20201112,60.0,462000.0,500.0,7700.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,162.0,1,9,3,2,2,9,5,5,8,1696,14,3,99,20180905,183.0,8125200.0,5000.0,44400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23.0,1,6,0,5,2,5,3,5,2,1369,14,1,1,20201127,1.0,198500.0,500.0,198500.0
752064,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103.0,1,6,0,5,2,5,3,5,2,443,14,99,99,20200731,50.0,372000.0,5000.0,7440.0
752065,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858.0,1,4,4,2,2,3,4,3,8,1279,14,3,99,20170704,75057.0,3677793.0,100.0,49.0
752066,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11.0,1,4,4,2,2,3,4,3,8,2374,14,2,99,20201215,96.0,11424000.0,2500.0,119000.0


In [24]:
df.to_csv('../data/train_test.csv',index=False)

In [25]:
train_label.to_csv('../data/train_label.csv',index=False)

In [26]:
df.columns = ['계좌ID','종목코드','매수일자','과거 보유일','성별','연령대',
              '투자성향','자산구간','주거래상품군','Life Stage','서비스등급','총 투자기간','주거래업종구분',
             '종목한글명','종목업종','시가총액규모유형','시장구분','기준일자','잔고수량','잔고금액','액면가','주당 거래가']

# 3.Feature Making

In [27]:
feature = pd.DataFrame()

#### byn_dt 해당 년도 마지막 날에서 빼기 hist_d1

In [28]:
df['매수일자_연'] = df['매수일자'].astype('str').apply(lambda x: x[:4]) + '1231'
df['매수일자_연'] = pd.to_datetime(df['매수일자_연'],format='%Y%m%d')
df['매수일자'] = pd.to_datetime(df['매수일자'],format='%Y%m%d')
df

,계좌ID,종목코드,매수일자,과거 보유일,성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액규모유형,시장구분,기준일자,잔고수량,잔고금액,액면가,주당 거래가,매수일자_연
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,5.0,1,9,3,2,2,9,5,5,8,101,14,1,1,20180726,274.0,11782000.0,5000.0,43000.0,2018-12-31
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,40.0,1,9,3,2,2,9,5,5,8,1361,14,1,1,20180131,2.0,4990000.0,5000.0,2495000.0,2018-12-31
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,2.0,1,9,3,2,2,9,5,5,8,2530,14,2,99,20180517,786.0,14619600.0,1000.0,18600.0,2018-12-31
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,11.0,1,9,3,2,2,9,5,5,8,1968,14,2,1,20201112,60.0,462000.0,500.0,7700.0,2020-12-31
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,162.0,1,9,3,2,2,9,5,5,8,1696,14,3,99,20180905,183.0,8125200.0,5000.0,44400.0,2018-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,2020-11-27,23.0,1,6,0,5,2,5,3,5,2,1369,14,1,1,20201127,1.0,198500.0,500.0,198500.0,2020-12-31
752064,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,2020-07-31,103.0,1,6,0,5,2,5,3,5,2,443,14,99,99,20200731,50.0,372000.0,5000.0,7440.0,2020-12-31
752065,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,2017-07-04,858.0,1,4,4,2,2,3,4,3,8,1279,14,3,99,20170704,75057.0,3677793.0,100.0,49.0,2017-12-31
752066,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,2020-12-15,11.0,1,4,4,2,2,3,4,3,8,2374,14,2,99,20201215,96.0,11424000.0,2500.0,119000.0,2020-12-31


In [29]:
feature['byn_dt_last'] = df['매수일자_연'] - df['매수일자']
feature['byn_dt_last'] = feature['byn_dt_last'].dt.days
bdl = []
for i in range(len(feature['byn_dt_last'])):
    if feature['byn_dt_last'][i] < df['과거 보유일'][i]:
        H_date = df_date[df_date.날짜 >= df['매수일자'][i]][df_date.날짜 <= df['매수일자_연'][i]].groupby('개장')['날짜'].count()[0]
        bdl.append(feature['byn_dt_last'][i]-H_date)
    else:
        bdl.append(df['과거 보유일'][i])
feature['byn_dt_last'] = bdl
feature

,byn_dt_last
0,5.0
1,40.0
2,2.0
3,11.0
4,76.0
...,...
752063,23.0
752064,103.0
752065,119.0
752066,11.0


#### 총 투자기간 대비 몇프로 차지 hist_d

In [30]:
df['총투자기간_h'] = df['총 투자기간'].apply(lambda x: 182 if x == 1 else (364 if x == 2 else (365*3-1 if x == 3 else (
                    365*5-1 if x == 4 else(365*10-1 if x==5 else 365*20 )))))

In [31]:
df['총투자기간_l'] = df['총 투자기간'].apply(lambda x: 30 if x == 1 else (185 if x == 2 else (365 if x == 3 else (
                    365*3-1 if x == 4 else(365*5-1 if x==5 else 365*10 )))))

In [32]:
df['총투자기간_m'] = df['총 투자기간'].apply(lambda x: 90 if x == 1 else (270 if x == 2 else (365*2 if x == 3 else (
                    365*4 if x == 4 else(365*7 if x==5 else 365*15 )))))

In [33]:
feature['hist/tot_h'] = df['과거 보유일']/df['총투자기간_h']
feature['hist/tot_l'] = df['과거 보유일']/df['총투자기간_l']
feature['hist/tot_m'] = df['과거 보유일']/df['총투자기간_m']
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m
0,5.0,0.001370,0.002741,0.001957
1,40.0,0.010962,0.021930,0.015656
2,2.0,0.000548,0.001096,0.000783
3,11.0,0.003015,0.006031,0.004305
4,76.0,0.044396,0.088816,0.063405
...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002
752064,103.0,0.028227,0.056469,0.040313
752065,119.0,0.784278,2.350685,1.175342
752066,11.0,0.010055,0.030137,0.015068


#### 총 투자기간 대비 몇프로 차지 byn_dt_last

In [34]:
feature['byn_last_h'] = feature['byn_dt_last']/df['총투자기간_h']
feature['byn_last_l'] = feature['byn_dt_last']/df['총투자기간_l']
feature['byn_last_m'] = feature['byn_dt_last']/df['총투자기간_m']
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746
...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068


#### hist Feature 만들기

In [35]:
Hold_hist_t = pd.DataFrame(hist.groupby(['act_id','iem_cd'])['bse_dt'].count()).reset_index(drop=False)
Hold_hist_t.columns = ['계좌ID','종목코드','hold_hist_t']
df = pd.merge(df,Hold_hist_t,on=['계좌ID','종목코드'],how='left')
feature['hold_hist_t'] = df['hold_hist_t']
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4
...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6


In [36]:
Hold_hist_t = pd.DataFrame(hist.groupby(['act_id','iem_cd'])['bnc_qty'].sum()).reset_index(drop=False)
Hold_hist_t.columns = ['계좌ID','종목코드','bnc_qty_s']
df = pd.merge(df,Hold_hist_t,on=['계좌ID','종목코드'],how='left')
feature['bnc_qty_s'] = df['bnc_qty_s']
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t,bnc_qty_s
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6,836.0
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9,2094.0
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3,986.0
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2,60.0
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4,329.0
...,...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6,17.0
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9,3980.0
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1,75057.0
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6,638.0


In [37]:
Hold_hist_t = pd.DataFrame(hist.groupby(['act_id','iem_cd'])['tot_aet_amt'].sum()).reset_index(drop=False)
Hold_hist_t.columns = ['계좌ID','종목코드','tot_aet_amt_s']
df = pd.merge(df,Hold_hist_t,on=['계좌ID','종목코드'],how='left')
feature['tot_aet_amt_s'] = df['tot_aet_amt_s']
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t,bnc_qty_s,tot_aet_amt_s
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6,836.0,33585800.0
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9,2094.0,177414100.0
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3,986.0,18499600.0
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2,60.0,462000.0
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4,329.0,12863700.0
...,...,...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6,17.0,3391500.0
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9,3980.0,30597600.0
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1,75057.0,3677793.0
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6,638.0,80874500.0


#### 자산대비 거래 금액

In [38]:
df['총자산_h'] = df['자산구간'].apply(lambda x: 10000000 if x == 1 else (30000000 if x == 2 else (50000000 if x == 3 else (
                    100000000 if x == 4 else(300000000 if x==5 else 500000000 )))))

In [39]:
df['총자산_l'] = df['자산구간'].apply(lambda x: 1000000 if x == 1 else (10000000 if x == 2 else (30000000 if x == 3 else (
                    50000000 if x == 4 else(100000000 if x==5 else 300000000 )))))

In [40]:
df['총자산_m'] = df['자산구간'].apply(lambda x: 5000000 if x == 1 else (20000000 if x == 2 else (40000000 if x == 3 else (
                    75000000 if x == 4 else(200000000 if x==5 else 400000000 )))))

In [41]:
feature['tot_aet/aet_stn_h'] = df['잔고금액']/df['총자산_h']
feature['tot_aet/aet_stn_l'] = df['잔고금액']/df['총자산_l']
feature['tot_aet/aet_stn_m'] = df['잔고금액']/df['총자산_m']

In [42]:
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t,bnc_qty_s,tot_aet_amt_s,tot_aet/aet_stn_h,tot_aet/aet_stn_l,tot_aet/aet_stn_m
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6,836.0,33585800.0,0.392733,1.178200,0.589100
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9,2094.0,177414100.0,0.166333,0.499000,0.249500
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3,986.0,18499600.0,0.487320,1.461960,0.730980
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2,60.0,462000.0,0.015400,0.046200,0.023100
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4,329.0,12863700.0,0.270840,0.812520,0.406260
...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6,17.0,3391500.0,0.000662,0.001985,0.000992
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9,3980.0,30597600.0,0.001240,0.003720,0.001860
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1,75057.0,3677793.0,0.122593,0.367779,0.183890
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6,638.0,80874500.0,0.380800,1.142400,0.571200


#### 거래금액 대비 수익기여도

In [43]:
df['수익기여도'] = df['서비스등급'].apply(lambda x: 5000000 if x == 1 else (3000000 if x == 2 else (1000000 if x == 3 else (
                    500000 if x == 4 else(100000 if x==5 or x==99 else 10000 )))))

In [44]:
feature['profit/tot_aet'] = df['수익기여도']/df['잔고금액']

In [45]:
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t,bnc_qty_s,tot_aet_amt_s,tot_aet/aet_stn_h,tot_aet/aet_stn_l,tot_aet/aet_stn_m,profit/tot_aet
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6,836.0,33585800.0,0.392733,1.178200,0.589100,0.008488
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9,2094.0,177414100.0,0.166333,0.499000,0.249500,0.020040
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3,986.0,18499600.0,0.487320,1.461960,0.730980,0.006840
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2,60.0,462000.0,0.015400,0.046200,0.023100,0.216450
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4,329.0,12863700.0,0.270840,0.812520,0.406260,0.012307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6,17.0,3391500.0,0.000662,0.001985,0.000992,5.037783
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9,3980.0,30597600.0,0.001240,0.003720,0.001860,2.688172
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1,75057.0,3677793.0,0.122593,0.367779,0.183890,0.135951
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6,638.0,80874500.0,0.380800,1.142400,0.571200,0.043768


#### 거래량 대비 수익기여도

In [46]:
tr = train_data.copy()
te = test.copy()

In [47]:
tr_passbook = pd.DataFrame(train_data.groupby('act_id')['bnc_qty'].sum()).reset_index()
te_passbook = pd.DataFrame(test_data.groupby('act_id')['bnc_qty'].sum()).reset_index()
tr_passbook.columns = ['act_id','계좌별전체거래량']
te_passbook.columns = ['act_id','계좌별전체거래량']
tr = pd.merge(tr,tr_passbook,on='act_id',how='left')
te = pd.merge(te,te_passbook,on='act_id',how='left')
tr_te = pd.concat([tr,te]).reset_index(drop=True)
tr_te

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p,계좌별전체거래량,submit_id,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,5.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,101.0,14.0,1.0,1.0,20180726.0,274.0,11782000.0,5000.0,43000.0,36209.0,NaN,NaN
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,40.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,1361.0,14.0,1.0,1.0,20180131.0,2.0,4990000.0,5000.0,2495000.0,36209.0,NaN,NaN
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,2.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,2530.0,14.0,2.0,99.0,20180517.0,786.0,14619600.0,1000.0,18600.0,36209.0,NaN,NaN
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,11.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,1968.0,14.0,2.0,1.0,20201112.0,60.0,462000.0,500.0,7700.0,36209.0,NaN,NaN
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,162.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,1696.0,14.0,3.0,99.0,20180905.0,183.0,8125200.0,5000.0,44400.0,36209.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,IDX70592,0.0
752064,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,IDX70593,0.0
752065,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75203.0,IDX70594,0.0
752066,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75203.0,IDX70595,0.0


In [48]:
df['계좌별전체거래량'] = tr_te['계좌별전체거래량']

In [49]:
feature['profit/tot_bnc_qty'] = (df['수익기여도']*df['잔고수량'])/df['계좌별전체거래량']

In [50]:
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t,bnc_qty_s,tot_aet_amt_s,tot_aet/aet_stn_h,tot_aet/aet_stn_l,tot_aet/aet_stn_m,profit/tot_aet,profit/tot_bnc_qty
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6,836.0,33585800.0,0.392733,1.178200,0.589100,0.008488,756.717943
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9,2094.0,177414100.0,0.166333,0.499000,0.249500,0.020040,5.523489
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3,986.0,18499600.0,0.487320,1.461960,0.730980,0.006840,2170.731034
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2,60.0,462000.0,0.015400,0.046200,0.023100,0.216450,165.704659
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4,329.0,12863700.0,0.270840,0.812520,0.406260,0.012307,505.399210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6,17.0,3391500.0,0.000662,0.001985,0.000992,5.037783,3937.007874
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9,3980.0,30597600.0,0.001240,0.003720,0.001860,2.688172,196850.393701
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1,75057.0,3677793.0,0.122593,0.367779,0.183890,0.135951,499029.294044
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6,638.0,80874500.0,0.380800,1.142400,0.571200,0.043768,638.272409


#### 전체 거래 수 대비 현 거래 수 

In [51]:
feature['bnc_qty/tot_bnc_qty'] = df['잔고수량']/df['계좌별전체거래량']

In [52]:
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t,bnc_qty_s,tot_aet_amt_s,tot_aet/aet_stn_h,tot_aet/aet_stn_l,tot_aet/aet_stn_m,profit/tot_aet,profit/tot_bnc_qty,bnc_qty/tot_bnc_qty
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6,836.0,33585800.0,0.392733,1.178200,0.589100,0.008488,756.717943,0.007567
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9,2094.0,177414100.0,0.166333,0.499000,0.249500,0.020040,5.523489,0.000055
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3,986.0,18499600.0,0.487320,1.461960,0.730980,0.006840,2170.731034,0.021707
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2,60.0,462000.0,0.015400,0.046200,0.023100,0.216450,165.704659,0.001657
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4,329.0,12863700.0,0.270840,0.812520,0.406260,0.012307,505.399210,0.005054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6,17.0,3391500.0,0.000662,0.001985,0.000992,5.037783,3937.007874,0.003937
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9,3980.0,30597600.0,0.001240,0.003720,0.001860,2.688172,196850.393701,0.196850
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1,75057.0,3677793.0,0.122593,0.367779,0.183890,0.135951,499029.294044,0.998059
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6,638.0,80874500.0,0.380800,1.142400,0.571200,0.043768,638.272409,0.001277


#### 전체 잔고금액 대비 잔고금액

In [53]:
tr_Balance_amount = pd.DataFrame(train_data.groupby('act_id')['tot_aet_amt'].sum()).reset_index()
te_Balance_amount = pd.DataFrame(test_data.groupby('act_id')['tot_aet_amt'].sum()).reset_index()
tr_Balance_amount.columns = ['act_id','계좌별전체잔고금액']
te_Balance_amount.columns = ['act_id','계좌별전체잔고금액']
tr = pd.merge(tr,tr_Balance_amount,on='act_id',how='left')
te = pd.merge(te,te_Balance_amount,on='act_id',how='left')
tr_te = pd.concat([tr,te]).reset_index(drop=True)
tr_te

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p,계좌별전체거래량,계좌별전체잔고금액,submit_id,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,5.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,101.0,14.0,1.0,1.0,20180726.0,274.0,11782000.0,5000.0,43000.0,36209.0,383002830.0,NaN,NaN
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,40.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,1361.0,14.0,1.0,1.0,20180131.0,2.0,4990000.0,5000.0,2495000.0,36209.0,383002830.0,NaN,NaN
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,2.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,2530.0,14.0,2.0,99.0,20180517.0,786.0,14619600.0,1000.0,18600.0,36209.0,383002830.0,NaN,NaN
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,11.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,1968.0,14.0,2.0,1.0,20201112.0,60.0,462000.0,500.0,7700.0,36209.0,383002830.0,NaN,NaN
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,162.0,1.0,9.0,3.0,2.0,2.0,9.0,5.0,5.0,8.0,1696.0,14.0,3.0,99.0,20180905.0,183.0,8125200.0,5000.0,44400.0,36209.0,383002830.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,4740000.0,IDX70592,0.0
752064,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254.0,4740000.0,IDX70593,0.0
752065,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75203.0,26501793.0,IDX70594,0.0
752066,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75203.0,26501793.0,IDX70595,0.0


In [54]:
df['계좌별전체잔고금액'] = tr_te['계좌별전체잔고금액']
df

,계좌ID,종목코드,매수일자,과거 보유일,성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액규모유형,시장구분,기준일자,잔고수량,잔고금액,액면가,주당 거래가,매수일자_연,총투자기간_h,총투자기간_l,총투자기간_m,hold_hist_t,bnc_qty_s,tot_aet_amt_s,총자산_h,총자산_l,총자산_m,수익기여도,계좌별전체거래량,계좌별전체잔고금액
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,5.0,1,9,3,2,2,9,5,5,8,101,14,1,1,20180726,274.0,11782000.0,5000.0,43000.0,2018-12-31,3649,1824,2555,6,836.0,33585800.0,30000000,10000000,20000000,100000,36209.0,383002830.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,40.0,1,9,3,2,2,9,5,5,8,1361,14,1,1,20180131,2.0,4990000.0,5000.0,2495000.0,2018-12-31,3649,1824,2555,9,2094.0,177414100.0,30000000,10000000,20000000,100000,36209.0,383002830.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,2.0,1,9,3,2,2,9,5,5,8,2530,14,2,99,20180517,786.0,14619600.0,1000.0,18600.0,2018-12-31,3649,1824,2555,3,986.0,18499600.0,30000000,10000000,20000000,100000,36209.0,383002830.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,11.0,1,9,3,2,2,9,5,5,8,1968,14,2,1,20201112,60.0,462000.0,500.0,7700.0,2020-12-31,3649,1824,2555,2,60.0,462000.0,30000000,10000000,20000000,100000,36209.0,383002830.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,162.0,1,9,3,2,2,9,5,5,8,1696,14,3,99,20180905,183.0,8125200.0,5000.0,44400.0,2018-12-31,3649,1824,2555,4,329.0,12863700.0,30000000,10000000,20000000,100000,36209.0,383002830.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,2020-11-27,23.0,1,6,0,5,2,5,3,5,2,1369,14,1,1,20201127,1.0,198500.0,500.0,198500.0,2020-12-31,3649,1824,2555,6,17.0,3391500.0,300000000,100000000,200000000,1000000,254.0,4740000.0
752064,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,2020-07-31,103.0,1,6,0,5,2,5,3,5,2,443,14,99,99,20200731,50.0,372000.0,5000.0,7440.0,2020-12-31,3649,1824,2555,9,3980.0,30597600.0,300000000,100000000,200000000,1000000,254.0,4740000.0
752065,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,2017-07-04,858.0,1,4,4,2,2,3,4,3,8,1279,14,3,99,20170704,75057.0,3677793.0,100.0,49.0,2017-12-31,1094,365,730,1,75057.0,3677793.0,30000000,10000000,20000000,500000,75203.0,26501793.0
752066,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,2020-12-15,11.0,1,4,4,2,2,3,4,3,8,2374,14,2,99,20201215,96.0,11424000.0,2500.0,119000.0,2020-12-31,1094,365,730,6,638.0,80874500.0,30000000,10000000,20000000,500000,75203.0,26501793.0


In [55]:
feature['aet_amt/tot_aet'] = df['잔고금액']/df['계좌별전체잔고금액']

In [56]:
feature

,byn_dt_last,hist/tot_h,hist/tot_l,hist/tot_m,byn_last_h,byn_last_l,byn_last_m,hold_hist_t,bnc_qty_s,tot_aet_amt_s,tot_aet/aet_stn_h,tot_aet/aet_stn_l,tot_aet/aet_stn_m,profit/tot_aet,profit/tot_bnc_qty,bnc_qty/tot_bnc_qty,aet_amt/tot_aet
0,5.0,0.001370,0.002741,0.001957,0.001370,0.002741,0.001957,6,836.0,33585800.0,0.392733,1.178200,0.589100,0.008488,756.717943,0.007567,0.030762
1,40.0,0.010962,0.021930,0.015656,0.010962,0.021930,0.015656,9,2094.0,177414100.0,0.166333,0.499000,0.249500,0.020040,5.523489,0.000055,0.013029
2,2.0,0.000548,0.001096,0.000783,0.000548,0.001096,0.000783,3,986.0,18499600.0,0.487320,1.461960,0.730980,0.006840,2170.731034,0.021707,0.038171
3,11.0,0.003015,0.006031,0.004305,0.003015,0.006031,0.004305,2,60.0,462000.0,0.015400,0.046200,0.023100,0.216450,165.704659,0.001657,0.001206
4,76.0,0.044396,0.088816,0.063405,0.020828,0.041667,0.029746,4,329.0,12863700.0,0.270840,0.812520,0.406260,0.012307,505.399210,0.005054,0.021214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,23.0,0.006303,0.012610,0.009002,0.006303,0.012610,0.009002,6,17.0,3391500.0,0.000662,0.001985,0.000992,5.037783,3937.007874,0.003937,0.041878
752064,103.0,0.028227,0.056469,0.040313,0.028227,0.056469,0.040313,9,3980.0,30597600.0,0.001240,0.003720,0.001860,2.688172,196850.393701,0.196850,0.078481
752065,119.0,0.784278,2.350685,1.175342,0.108775,0.326027,0.163014,1,75057.0,3677793.0,0.122593,0.367779,0.183890,0.135951,499029.294044,0.998059,0.138775
752066,11.0,0.010055,0.030137,0.015068,0.010055,0.030137,0.015068,6,638.0,80874500.0,0.380800,1.142400,0.571200,0.043768,638.272409,0.001277,0.431065


In [57]:
feature.to_csv('../data/feature_ygy.csv',index=False)